In [1]:
import os

os.environ["WANDB_PROJECT"] = "hidden_capacity_reasoning"
from transformers import Qwen2ForCausalLM, Qwen2Model, AutoTokenizer, BitsAndBytesConfig
import torch
from trl import (
    ModelConfig,
    ScriptArguments,
    SFTConfig,
    SFTTrainer,
    TrlParser,
    get_kbit_device_map,
)

from datasets import load_dataset
from tqdm import tqdm
from hidden_capacity_reasoning.utils import (
    generate_train_examples,
    pad_train_examples,
    tokenize_single_turn,
)
from datasets import Dataset
import gc
import types

# need for auto SFTTrainer patch(possible increase speed)
from unsloth import is_bfloat16_supported
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from hidden_capacity_reasoning.utils import (
    EOS_TOKEN_ID,
    TEXT_TOKEN_ID,
    WINDOW_SIZE,
    VISION_START,
    VISION_END,
    find_all_linear_names_v3,
)

import time
from datetime import datetime


from hidden_capacity_reasoning.models import (
    Qwen2ForCausalLMCompressionV1,
    Qwen2ModelEmbedPoolerV1,
    Qwen2ForCausalLMCompressionV2,
    Qwen2ModelEmbedPoolerV2,
    Qwen2ForCausalLMCompressionV3,
    Qwen2ModelEmbedPoolerV3,
)

# model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model_name = "my_r1_model_v3"
model = Qwen2ForCausalLMCompressionV3.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    # torch_dtype=torch.float32,
    device_map={"": 0},
    attn_implementation="flash_attention_2",
)
device = "cuda"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.model.requires_grad_(False)
# model = model.to(torch.bfloat16)

# temp_model = Qwen2ModelEmbedPoolerV3.from_pretrained(
#     model_name,
#     attn_implementation="flash_attention_2",
#     torch_dtype=torch.bfloat16,
#     device_map={"": 0},
#     # quantization_config=BitsAndBytesConfig(load_in_4bit=True),
# )
# print(
#     model.embed_pooler.load_state_dict(
#         temp_model.state_dict(),
#         strict=False,
#     ),
# )
# temp_model = temp_model.cpu()
# del temp_model
# gc.collect()
# torch.cuda.empty_cache()

dataset = load_dataset("dim/open_orca_905_DeepSeek-R1-Distill-Qwen-1.5B")
dataset = dataset["train"]
dataset = dataset.train_test_split(test_size=10, seed=42)

# test pass
tokenize_single_turn(
    question=dataset["train"][0]["question"],
    answer=dataset["train"][0]["answer"],
    tokenizer=tokenizer,
)
train_examples = [
    tokenize_single_turn(tokenizer=tokenizer, **item)
    for item in tqdm(dataset["train"].to_list()[:3])
]

prepared_train_examples = []
for item in tqdm(train_examples):
    for example in generate_train_examples(
        dataset_batch=[item],
        window_size=WINDOW_SIZE,
    ):
        prepared_train_examples.append(example)

print(
    "max_len",
    max([len(item["original_tokens"]) for item in prepared_train_examples]),
)

new_dataset = Dataset.from_list(prepared_train_examples)
print(dataset)


def collate_fn(batch):
    padded_batch = pad_train_examples(
        train_examples=batch,
        tokenizer=tokenizer,
    )
    padded_batch = {
        "replaced_original_tokens": padded_batch["replaced_original_tokens"][
            "input_ids"
        ],
        "compressed_input_ids": padded_batch["compressed_input_ids"]["input_ids"],
        "original_tokens": padded_batch["original_tokens"]["input_ids"],
        "attention_mask": padded_batch["compressed_input_ids"]["attention_mask"],
        "labels": padded_batch["compressed_input_ids"]["input_ids"],
        "content_compression_mask": padded_batch["content_compression_mask"][
            "input_ids"
        ],
    }
    for key in padded_batch.keys():
        padded_batch[key] = torch.tensor(padded_batch[key])
    skip_ids = [
        TEXT_TOKEN_ID,
        EOS_TOKEN_ID,
        VISION_START,
        VISION_END,
    ]
    for skip_id in skip_ids:
        padded_batch["labels"][padded_batch["labels"] == skip_id] = -100
    # часть инпута от пользователя
    last_index = (padded_batch["content_compression_mask"] == 1).long().nonzero()[-1][1]
    padded_batch["labels"][:, :last_index][
        padded_batch["content_compression_mask"][:, :last_index] == 1
    ] = -100
    # print(padded_batch)
    return padded_batch


peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.0,
    bias="none",
    target_modules=find_all_linear_names_v3(model=model),
    modules_to_save=[
        "embed_pooler.model.embed_tokens",
        "embed_pooler.weight_pooler",
    ],
)

formatted_date = datetime.fromtimestamp(time.time()).strftime("%Y_%m_%d_%H_%M_%S_%f")
model.embed_pooler = prepare_model_for_kbit_training(model.embed_pooler)
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=new_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        warmup_steps=5,
        num_train_epochs=1,  # 90,  # Set this for 1 full training run.
        # num_train_epochs=90,  # Set this for 1 full training run.
        # max_steps=10000,
        learning_rate=1e-4,
        bf16=True,
        # fp16=model.dtype == torch.float16,
        logging_steps=8,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir=f"outputs/{formatted_date}",
        # report_to="wandb",
        report_to="none",
        remove_unused_columns=False,
        dataset_kwargs={"skip_prepare_dataset": True},
        # gradient_checkpointing=True,
        save_steps=10000,
        run_name=formatted_date,
    ),
)
trainer.train()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 38.69it/s]


max_len 580
DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 895
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 10
    })
})
trainable params: 254,197,760 || all params: 3,577,359,360 || trainable%: 7.1057


/tmp/ipykernel_585463/158584843.py:174: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
8,0.000000
16,0.000000
24,0.000000
32,0.000000
40,0.000000
48,0.000000
56,0.000000
64,0.000000
72,0.000000
80,0.000000


TrainOutput(global_step=169, training_loss=0.0, metrics={'train_runtime': 61.8415, 'train_samples_per_second': 10.931, 'train_steps_per_second': 2.733, 'total_flos': 0.0, 'train_loss': 0.0})

In [8]:
dataset[0]

{'problem': 'An elephant and a lion are currently 1 mile apart. The elephant runs directly away from the lion at 19 miles per hour, while the lion runs directly towards the elephant at 24 miles per hour.  How many minutes will it take for the lion to catch the elephant?',
 'solution': 'Every hour, the lion runs 24 miles while the elephant runs 19.  Thus, the distance between the two animals closes at a rate of 5 miles every hour.  The lion catches the elephant after this distance has closed 1 mile, which takes $\\frac{1}{5}$ hours to do, or $\\frac{1}{5}\\cdot 60 = \\boxed{12}$ minutes.',
 'answer': '12',
 'subject': 'Prealgebra',
 'level': 5,
 'unique_id': 'test/prealgebra/1820.json',
 'model_answer': "Okay, so I have this problem here where an elephant and a lion are one mile apart. The elephant is running directly away from the lion at 19 miles per hour, and the lion is running towards the elephant at 24 miles per hour. I need to figure out how many minutes it will take for the lion

In [4]:
model.save_pretrained("my_r1_model_v3")

In [5]:
tokenizer.save_pretrained("my_r1_model_v3")

('my_r1_model_v3/tokenizer_config.json',
 'my_r1_model_v3/special_tokens_map.json',
 'my_r1_model_v3/tokenizer.json')

In [ ]:
trainer.model

In [ ]:
trainer.model.base_model.embed_pooler

In [3]:
model.to(torch.bfloat16)
model.embed_pooler(
    torch.randn(
        2,
        2,
        1536,
        device="cuda",
        dtype=torch.bfloat16,
    )
).shape

torch.Size([2, 1, 1536])

In [3]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name}, Requires Gradient: {param.requires_grad}")

Layer: embed_pooler.model.embed_tokens.modules_to_save.default.weight, Requires Gradient: True
Layer: embed_pooler.model.layers.0.self_attn.q_proj.lora_A.default.weight, Requires Gradient: True
Layer: embed_pooler.model.layers.0.self_attn.q_proj.lora_B.default.weight, Requires Gradient: True
Layer: embed_pooler.model.layers.0.self_attn.k_proj.lora_A.default.weight, Requires Gradient: True
Layer: embed_pooler.model.layers.0.self_attn.k_proj.lora_B.default.weight, Requires Gradient: True
Layer: embed_pooler.model.layers.0.self_attn.v_proj.lora_A.default.weight, Requires Gradient: True
Layer: embed_pooler.model.layers.0.self_attn.v_proj.lora_B.default.weight, Requires Gradient: True
Layer: embed_pooler.model.layers.0.self_attn.o_proj.lora_A.default.weight, Requires Gradient: True
Layer: embed_pooler.model.layers.0.self_attn.o_proj.lora_B.default.weight, Requires Gradient: True
Layer: embed_pooler.model.layers.0.mlp.gate_proj.lora_A.default.weight, Requires Gradient: True
Layer: embed_pool

In [2]:
for name, param in trainer.model.named_parameters():
    if param.requires_grad:
        print(f"Layer: {name}, Requires Gradient: {param.requires_grad}")

Layer: base_model.model.embed_pooler.model.embed_tokens.modules_to_save.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.q_proj.lora_A.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.q_proj.lora_B.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.k_proj.lora_A.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.k_proj.lora_B.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.v_proj.lora_A.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.v_proj.lora_B.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.o_proj.lora_A.default.weight, Requires Gradient: True
Layer: base_model.model.embed_pooler.model.layers.0.self_attn.o_proj.lora_B.

In [3]:
model = trainer.model

In [ ]:
dataset

In [1]:
import torch
import random
import numpy as np

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
torch.set_grad_enabled(False)

from peft import PeftModel
from hidden_capacity_reasoning.models import (
    Qwen2ForCausalLMCompressionV1,
    Qwen2ModelEmbedPoolerV1,
    Qwen2ForCausalLMCompressionV2,
    Qwen2ModelEmbedPoolerV2,
)
import torch
from transformers import AutoTokenizer

# model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
model_name = "r1_compressor_v2"
model = Qwen2ForCausalLMCompressionV2.from_pretrained(
    model_name,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float32,
    device_map={"": 0},
    # attn_implementation="flash_attention_2",
    attn_implementation="sdpa",
)
model = PeftModel.from_pretrained(
    model,
    # "outputs/2025_04_19_17_17_34_493839/checkpoint-210000",
    # "outputs/2025_04_19_17_17_34_493839/checkpoint-50000",
    # "outputs/2025_04_21_19_23_11_642509/checkpoint-10000",
    # "outputs/2025_04_22_01_43_43_347583/checkpoint-90000",
    # "outputs/2025_04_22_01_43_43_347583/checkpoint-10000",
    # 'outputs/2025_04_23_00_46_10_062896/checkpoint-2794'
    "outputs/2025_04_29_21_24_17_071961/checkpoint-588",
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from datasets import load_dataset

dataset = load_dataset(
    # "dim/hendrycks_math_train_12k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096"
    # "dim/hendrycks_math_test_500_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
    "dim/hendrycks_math_train_1k_DeepSeek-R1-Distill-Qwen-1.5B_max_len_4096_greedy"
)

dataset = dataset["train"].train_test_split(
    test_size=250,
    # test_size=1,
    seed=42,
)
dataset = dataset["test"].filter(lambda x: x["model_answer"].count("</think>") == 1)

from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)

correct_dataset = []

for pos, item in enumerate(dataset):
    try:
        answer = dataset_answer_filter(item["answer"])
        model_answer = model_answer_filter(item["model_answer"])
        # print(answer, model_answer)
        # break
        if is_equiv(answer, model_answer):
            correct_dataset.append(item)
    except:
        pass

len(dataset), len(correct_dataset), len(correct_dataset) / len(dataset)

'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'
'NoneType' object has no attribute 'group'


(149, 134, 0.8993288590604027)

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda"
# prompt = "how many wings has a bird?"
prompt = correct_dataset[0]["problem"]

base_prompt = open(
    "hidden_capacity_reasoning/evaluation/math_500/math_500_prompt"
).read()

messages = [
    {"role": "user", "content": base_prompt.format(question=prompt)},
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
print(text)
model_inputs = tokenizer(
    [text],
    return_tensors="pt",
    padding="longest",
    truncation=False,
    add_special_tokens=False,
).to(device)

with torch.no_grad():
    # generated_ids = model.generate(
    #     model_inputs.input_ids,
    #     max_new_tokens=1,
    #     do_sample=False,
    # )
    generated_ids = model.generate(
        # **model_inputs.input_ids,
        **model_inputs,
        max_new_tokens=4096,
        do_sample=False,
        # do_sample=not False,
        # temperature=0.6,
        # top_p=0.95,
    )
generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response

###

<｜begin▁of▁sentence｜><｜User｜>Problem: If $A=2+i$, $O=-4$, $P=-i$, and $S=2+4i$, find $A-O+P+S$.

Please reason step by step, and put your final answer within \boxed{}.<｜Assistant｜><think>



"Okay, so I have this problem here where I need to find the value of A minus O plus P plus S. The given values are A = 2 + i, O = -4, P = -i, and S = 2 + 4i. Hmm, let me write that down to make it clearer.\n\nFirst, let me write down each term:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nAnd the expression I need to compute is A - O + P + S. Hmm, okay. So, I think I need to substitute these values into the expression and then simplify it step by step.\n\nLet me start by substituting the values:\n\nA - O + P + S = (2 + i) - (-4) + (-i) + (2 + 4i)\n\nWait, hold on. Let me make sure I substitute correctly. So, A is 2 + i, O is -4, P is -i, and S is 2 + 4i. So, replacing each term:\n\n= (2 + i) - (-4) + (-i) + (2 + 4i)\n\nOkay, that looks right. Now, let me simplify this step by step. I think I should handle the operations one at a time.\n\nFirst, let's handle the subtraction of O, which is -(-4). Subtracting a negative is the same as adding a positive, so:\n\n(2 + i) - (-4) = (2 + i

In [4]:
correct_dataset[0]

{'problem': 'An elephant and a lion are currently 1 mile apart. The elephant runs directly away from the lion at 19 miles per hour, while the lion runs directly towards the elephant at 24 miles per hour.  How many minutes will it take for the lion to catch the elephant?',
 'solution': 'Every hour, the lion runs 24 miles while the elephant runs 19.  Thus, the distance between the two animals closes at a rate of 5 miles every hour.  The lion catches the elephant after this distance has closed 1 mile, which takes $\\frac{1}{5}$ hours to do, or $\\frac{1}{5}\\cdot 60 = \\boxed{12}$ minutes.',
 'answer': '12',
 'subject': 'Prealgebra',
 'level': 5,
 'unique_id': 'test/prealgebra/1820.json',
 'model_answer': "Okay, so I have this problem here where an elephant and a lion are one mile apart. The elephant is running directly away from the lion at 19 miles per hour, and the lion is running towards the elephant at 24 miles per hour. I need to figure out how many minutes it will take for the lion

In [5]:
response  # == dataset[0]['model_answer']

"Okay, so I have this problem here where I need to find the value of A minus O plus P plus S. The given values are A = 2 + i, O = -4, P = -i, and S = 2 + 4i. Hmm, let me write that down to make it clearer.\n\nFirst, let me write down each term:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nAnd the expression I need to compute is A - O + P + S. Hmm, okay. So, I think I need to substitute these values into the expression and then simplify it step by step.\n\nLet me start by substituting the values:\n\nA - O + P + S = (2 + i) - (-4) + (-i) + (2 + 4i)\n\nWait, hold on. Let me make sure I substitute correctly. So, A is 2 + i, O is -4, P is -i, and S is 2 + 4i. So, replacing each term:\n\n= (2 + i) - (-4) + (-i) + (2 + 4i)\n\nOkay, that looks right. Now, let me simplify this step by step. I think I should handle the operations one at a time.\n\nFirst, let's handle the subtraction of O, which is -(-4). Subtracting a negative is the same as adding a positive, so:\n\n(2 + i) - (-4) = (2 + i

In [ ]:
dataset[0]["model_answer"]

"Okay, so I have this problem here where I need to find the value of A minus O plus P plus S. The given values are A = 2 + i, O = -4, P = -i, and S = 2 + 4i. Hmm, let me write that down to make it clearer.\n\nFirst, let me write down each term:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nAnd the expression I need to compute is A - O + P + S. Hmm, okay. So, I think I need to substitute these values into the expression and then simplify it step by step.\n\nLet me start by substituting the values:\n\nA - O + P + S = (2 + i) - (-4) + (-i) + (2 + 4i)\n\nWait, hold on. Let me make sure I substitute correctly. So, A is 2 + i, O is -4, P is -i, and S is 2 + 4i. So, replacing each term:\n\n= (2 + i) - (-4) + (-i) + (2 + 4i)\n\nOkay, that looks right. Now, let me simplify this step by step. I think I should handle the operations one at a time.\n\nFirst, let's handle the subtraction of O, which is -(-4). Subtracting a negative is the same as adding a positive, so:\n\n(2 + i) - (-4) = (2 + i

### Greedy Generation

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda"
prompt = correct_dataset[:5][0]["problem"]

base_prompt = open(
    "hidden_capacity_reasoning/evaluation/math_500/math_500_prompt"
).read()

generated_tokens = tokenizer.apply_chat_template(
    [
        {"role": "user", "content": base_prompt.format(question=prompt)},
    ],
    tokenize=True,
    add_generation_prompt=True,
)
# initial_len = generated_tokens.shape
with torch.no_grad():
    generated_tokens = torch.tensor(generated_tokens).unsqueeze(0).cuda()
    generated_embeds = model.get_input_embeddings()(generated_tokens)
    max_steps = 1200
    past_key_values = None
    for step in range(max_steps):
        if step == 0:
            logits = model(
                inputs_embeds=generated_embeds,
                attention_mask=torch.ones(1, generated_embeds.shape[1]).long().cuda(),
                position_ids=torch.arange(generated_embeds.shape[1])
                .cuda()
                .unsqueeze(0),
                use_cache=True,
                past_key_values=None,
            )
            past_key_values = logits.past_key_values
            logits = logits.logits[:, -1, :].clone().float()
        else:
            logits = model(
                # input_ids=generated_tokens[-1][-1:].unsqueeze(0),
                inputs_embeds=generated_embeds[:, -1:, :],
                attention_mask=torch.ones(1, generated_embeds.shape[1]).long().cuda(),
                position_ids=torch.tensor(generated_embeds.shape[1] - 1)
                .reshape(1, 1)
                .cuda(),
                use_cache=True,
                past_key_values=past_key_values,
            )
            past_key_values = logits.past_key_values

            logits = logits.logits[:, -1, :].clone().float()

        top_token = logits.argmax(-1)[-1]
        top_token_embed = model.get_input_embeddings()(top_token)
        # print(top)
        generated_tokens = torch.cat([generated_tokens, top_token.reshape(1, 1)], dim=1)
        generated_embeds = torch.cat(
            [generated_embeds, top_token_embed.reshape(1, 1, -1)],
            dim=1,
        )
        # print(step, tokenizer.decode(generated_tokens[-1]))
    # break
print(tokenizer.decode(generated_tokens[-1]))
# break
embeds_generation_tokens = generated_tokens[-1]
###

<｜begin▁of▁sentence｜><｜User｜>Problem: An elephant and a lion are currently 1 mile apart. The elephant runs directly away from the lion at 19 miles per hour, while the lion runs directly towards the elephant at 24 miles per hour.  How many minutes will it take for the lion to catch the elephant?

Please reason step by step, and put your final answer within \boxed{}.<｜Assistant｜><think>
Okay, so I have this problem where an elephant and a lion are 1 mile apart. The elephant is running directly away from the lion at 19 miles per hour, and the lion is running towards the elephant at 24 miles per hour. I need to figure out how many minutes it will take for the lion to catch the elephant. Hmm, let me think about how to approach this.

First, I know that both the elephant and the lion are moving towards or away from each other. The elephant is moving away at 19 mph, and the lion is moving towards it at 24 mph. So, their relative speed is the difference between their speeds because they're mov

In [106]:
tokenizer.decode(generated_tokens[-1])

"<｜begin▁of▁sentence｜><｜User｜>Problem: An elephant and a lion are currently 1 mile apart. The elephant runs directly away from the lion at 19 miles per hour, while the lion runs directly towards the elephant at 24 miles per hour.  How many minutes will it take for the lion to catch the elephant?\n\nPlease reason step by step, and put your final answer within \\boxed{}.<｜Assistant｜><think>\nOkay, so I have this problem where an elephant and a lion are 1 mile apart. The elephant is running directly away from the lion at 19 miles per hour, and the lion is running towards the elephant at 24 miles per hour. I need to figure out how many minutes it will take for the lion to catch the elephant. Hmm, let me think about how to approach this.\n\nFirst, I know that both the elephant and the lion are moving towards or away from each other. The elephant is moving away at 19 mph, and the lion is moving towards it at 24 mph. So, their relative speed is the difference between their speeds because they

In [14]:
from transformers.generation import utils
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda"
dataset_pos = 10
prompt = correct_dataset[dataset_pos]["problem"]

base_prompt = open(
    "hidden_capacity_reasoning/evaluation/math_500/math_500_prompt"
).read()

generated_tokens = tokenizer.apply_chat_template(
    [
        {
            "role": "user",
            "content": base_prompt.format(question=prompt),
        },
    ],
    tokenize=True,
    add_generation_prompt=True,
)
generated_tokens = torch.tensor(generated_tokens).unsqueeze(0).cuda()
generated_embeds = model.get_input_embeddings()(generated_tokens)

generated_ids = model.generate(
    inputs_embeds=generated_embeds,
    max_new_tokens=1800,
    # max_new_tokens=5,
    do_sample=False,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [34]:
correct_dataset[dataset_pos]

{'problem': "Alicia's average on her five tests is 88 points. The score range for each test is 0 points to 100 points, inclusive. What is the lowest possible score that Alicia could have earned on one of the five tests?",
 'solution': "If Alicia's average score on her five tests is 88 points, then the sum of her scores must be $88 \\times 5 = 440$ points. If she earned   100 points on four of the tests, then she could have earned a score as low as $\\boxed{40\\text{ points}}$ on the other test.",
 'answer': '40\\text{ points}',
 'subject': 'Prealgebra',
 'level': 4,
 'unique_id': 'test/prealgebra/1697.json',
 'model_answer': "Okay, so I need to figure out the lowest possible score Alicia could have earned on one of her five tests. She has an average of 88 points, and each test is scored between 0 and 100, inclusive. Hmm, let me think about this step by step.\n\nFirst, I know that an average is calculated by taking the sum of all the test scores and dividing it by the number of tests. I

In [16]:
import json

example_id = correct_dataset[dataset_pos]["unique_id"].replace("/", "_")
with open(f"./temp/{example_id}", "w") as f:
    temp = {
        "generated_ids": generated_ids.tolist(),
        "input_ids": generated_tokens.tolist(),
    }
    json.dump(temp, f)

In [ ]:
tokenizer.encode(
    correct_dataset[dataset_pos]["model_answer"],
    add_special_tokens=False,
)

In [ ]:
tokenizer.decode(generated_ids[-1])

In [ ]:
tokenizer.decode(generated_tokens[-1])

In [ ]:
print(tokenizer.decode(generated_ids[-1]))

In [ ]:
print(tokenizer.decode(generated_tokens[-1]))

In [23]:
tokenizer.decode(generated_tokens[-1])

"<｜begin▁of▁sentence｜><｜User｜>An elephant and a lion are currently 1 mile apart. The elephant runs directly away from the lion at 19 miles per hour, while the lion runs directly towards the elephant at 24 miles per hour.  How many minutes will it take for the lion to catch the elephant?<｜Assistant｜><think>\nOkay, so I have this problem where an elephant and a lion are 1 mile apart. The elephant is running directly away from the lion at 19 miles per hour, and the lion is running towards the elephant at 24 miles per hour. I need to figure out how many minutes it will take for the lion to catch the elephant. Hmm, let me think about how to approach this.\n\nFirst, I know that both the elephant and the lion are moving towards or away from each other. The elephant is moving away at 19 mph, and the lion is moving towards the elephant at 24 mph. So, their speeds are different, which means the distance between them will be changing over time. I need to find the time it takes for the lion to clo

In [ ]:
# Okay, so I have this problem where an elephant and a lion are 1 mile apart. The elephant is running directly away from the lion at 19 miles per hour, and the lion is running towards the elephant at 24 miles per hour. I need to figure out how many minutes it will take for the lion to catch the elephant. Hmm, let me think about how to approach this.\n\nFirst, I know that both the elephant and the lion are moving towards or away from each other. The elephant is moving away at 19 mph, and the lion is moving towards the elephant at 24 mph. So, their speeds are different, which means the distance between them is changing over time. I need to find the time it takes for the lion to close that 1 mile gap.\n\nI remember that when two objects are moving towards each other, their relative speed is the sum of their individual speeds. But in this case, the elephant is moving away, so it's like the lion is chasing the elephant, but the elephant is moving away. So, maybe I should consider the relative speed between the lion and the elephant.\n\nWait, actually, since the elephant is moving away, the lion has to cover the initial distance plus any additional distance the elephant might cover while the lion is chasing. But since the lion is faster, it should eventually catch up. So, maybe I can model this as a relative speed problem.\n\nLet me denote the speed of the lion as \\( v_l = 24 \\) mph and the speed of the elephant as \\( v_e = 19 \\) mph. The initial distance between them is \\( d = 1 \\) mile.\n\nSince the lion is moving towards the elephant and the elephant is moving away, the relative speed at which the distance between them is decreasing is the difference between the lion's speed and the elephant's speed. So, the relative speed \\( v_{relative} = v_l - v_e = 24 - 19 = 5 \\) mph.\n\nWait, is that right? If the lion is moving towards the elephant and the elephant is moving away, then the lion is effectively closing the gap at a rate of 5 mph. So, the time it takes to close the 1 mile gap would be the initial distance divided by the relative speed.\n\nSo, time \\( t = \\frac{d}{v_{relative}} = \\frac{1}{5} \\) hours. But the question asks for the time in minutes, so I need to convert that.\n\nSince 1 hour is 60 minutes, \\( \\frac{1}{5} \\) hours is \\( \\frac{1}{5} \\times 60 = 12 \\) minutes. So, it should take 12 minutes for the lion to catch the elephant.\n\nWait, let me double-check that. If the lion is moving at 24 mph and the elephant is moving away at 19 mph, then the lion is gaining on the elephant at a rate of 5 mph. So, to cover the 1 mile gap, it would take \\( \\frac{1}{5} \\) hours, which is indeed 12 minutes. That seems correct.\n\nAlternatively, I can model this with equations. Let me set up a coordinate system where at time \\( t \\) hours, the position of the lion is \\( 24t \\) miles from its starting point, and the position of the elephant is \\( 1 + 19t \\) miles from the starting point of the lion. Wait, actually, hold on. If the elephant is moving away from the lion, then the distance between them is increasing. So, the position of the elephant is \\( 1 + 19t \\) miles from the starting point of the lion, and the position of the lion is \\( 24t \\) miles from the same starting point.\n\nWait, but initially, they are 1 mile apart. So, if the lion starts at position 0 and the elephant starts at position 1 mile, then the distance between them at time \\( t \\) is \\( |24t - (1 + 19t)| \\). We want this distance to be 0 when the lion catches the elephant.\n\nSo, setting up the equation:\n\n\\( 24t - (1 + 19t) = 0 \\)\n\nSimplify:\n\n\\( 24t - 19t - 1 = 0 \\)\n\n\\( 5t - 1 = 0 \\)\n\n\\( 5t = 1 \\)\n\n\\( t = \\frac{1}{5} \\) hours, which is 12 minutes. So, that confirms my earlier result.\n\nAlternatively, I can think about it in terms of distance covered. The lion needs to cover the initial 1 mile plus whatever distance the elephant covers in that time. But since the lion is faster, the time is determined by the relative speed.\n\nWait, another way to think about it is using the concept of relative velocity. The lion is moving towards the elephant at 24 mph, and the elephant is moving away at 19 mph. So, the relative speed is 24 - 19 = 5 mph. So, the lion is closing the gap at 5 mph. Therefore, the time to close 1 mile is 1/5 hours, which is 12 minutes.\n\nI think that's solid. I can't see any mistakes in this reasoning. So, I think the answer is 12 minutes.\n\n**Final Answer**\nThe lion will catch the elephant in \\boxed{12} minutes.\n</think>\n\nThe elephant and the lion are initially 1 mile apart. The elephant runs directly away from the lion at 19 miles per hour, while the lion runs directly towards the elephant at 24 miles per hour. To determine how long it will take for the lion to catch the elephant, we need to consider their relative speed.\n\nThe relative speed at which the lion is closing the gap is the difference between their speeds:\n\\[ v_{\\text{relative}} = v_l - v_e = 24 \\text{ mph} - 19 \\text{ mph} = 5 \\text{ mph} \\]\n\nThe time it takes to close the 1 mile gap is calculated by dividing the initial distance by the relative speed:\n\\[ t = \\frac{1 \\text{ mile}}{5 \\text{ mph}} = \\frac{1}{5} \\text{ hours} \\]\n\nConverting this time into minutes:\n\\[ \\frac{1}{5} \\text{ hours} \\times 60 \\text{ minutes per hour} = 12 \\text{ minutes} \\]\n\nThus, the lion will catch the elephant in \\boxed{12} minutes.

### Generate with compression

In [6]:
from transformers import AutoTokenizer
from hidden_capacity_reasoning.utils import (
    WINDOW_SIZE,
    VISION_START,
    VISION_END,
    EOS_TOKEN_ID,
)
import torch
import json

torch.manual_seed(0)

dataset_pos = 0
input_ids = correct_dataset[dataset_pos]["problem"]
print(input_ids)
print("===")
print("===")
base_prompt = open(
    "hidden_capacity_reasoning/evaluation/math_500/math_500_prompt"
).read()

input_ids = [
    tokenizer.apply_chat_template(
        [
            {
                "role": "user",
                "content": base_prompt.format(question=input_ids),
            },
        ],
        tokenize=True,
        add_generation_prompt=True,
    )
]

# input_ids = dataset_item["input_ids"]
# generated_ids = dataset_item["generated_ids"]
generated_ids = [
    tokenizer.encode(
        correct_dataset[dataset_pos]["model_answer"],
        add_special_tokens=False,
    )
]

tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda"

# generated_tokens = tokenizer.apply_chat_template(
#     [
#         # {"role": "user", "content": "how many wings has a bird?"},
#         {"role": "user", "content": example["question"]},
#     ],
#     tokenize=True,
#     add_generation_prompt=True,
# )
generated_tokens = input_ids

with torch.no_grad():
    start_embed = model.base_model.embed_pooler.model.get_input_embeddings()(
        # start_embed = model.base_model.embed_pooler.model.embed_tokens.modules_to_save.default(
        torch.tensor([[VISION_START]], device="cuda")
    )
    end_embed = model.base_model.embed_pooler.model.get_input_embeddings()(
        # end_embed = model.base_model.embed_pooler.model.embed_tokens.modules_to_save.default(
        torch.tensor([[VISION_END]], device="cuda")
    )
    input_ids = torch.tensor(input_ids).cuda()
    input_ids_embeds = model.get_input_embeddings()(input_ids)
    windows_amount = 100
    # windows_amount = 200
    # windows_amount = 300
    # windows_amount = 400
    # windows_amount = 500
    # windows_amount = 2
    next_true_tokens = torch.tensor(generated_ids, device="cuda")[
        :, : WINDOW_SIZE * windows_amount
    ]

    # next_true_tokens = torch.tensor(next_true_tokens, device="cuda")

    original_embeds = (
        # model.base_model.embed_pooler.model.get_input_embeddings()(next_true_tokens)
        # model.base_model.model.get_input_embeddings()(next_true_tokens)
        model.get_input_embeddings()(next_true_tokens)
    ).to(torch.float32)
    # ).to(torch.bfloat16)

    # compressed_part = model.base_model.embed_pooler(new_embeds_for_compression)
    new_embeds_for_compression = original_embeds.reshape(
        windows_amount, WINDOW_SIZE, -1
    )
    compressed_part = model.base_model.embed_pooler(new_embeds_for_compression)
    compressed_part = compressed_part.reshape(1, windows_amount, -1)
    # start_embed = torch.rand_like(start_embed)
    # compressed_part = torch.rand_like(compressed_part)
    # end_embed = torch.rand_like(end_embed)
    generated_embeds = torch.cat(
        [
            input_ids_embeds,
            start_embed,
            compressed_part,
            end_embed,
        ],
        dim=1,
    )
    print("COMPRESSED PART", tokenizer.decode(next_true_tokens[-1]))
    print("===")
    generated_ids_compressed = model.generate(
        inputs_embeds=generated_embeds,
        max_new_tokens=2800,
        # max_new_tokens=5,
        do_sample=False,
        # do_sample=True,
        # temperature=0.6,
        # top_p=0.95,
    )
    # break
print(tokenizer.decode(generated_ids_compressed[-1]))
# break
# embeds_generation_tokens = generated_tokens[-1]

If $A=2+i$, $O=-4$, $P=-i$, and $S=2+4i$, find $A-O+P+S$.
===
===


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


COMPRESSED PART Okay, so I have this problem here where I need to find the value of A minus O plus P plus S. The given values are A = 2 + i, O = -4, P = -i, and S = 2 + 4i. Hmm, let me write that down to make it clearer.

First, let me write down each term:

A = 2 + i

O = -4

P = -i

S = 2 + 4i

And the expression I need to compute is A - O + P + S. Hmm, okay. So, I think I need to substitute these values into the expression and then simplify it step by step.

Let me start by substituting the values:

A - O + P + S = (2 + i) - (-4) + (-i) + (2 + 4i)

Wait, hold on. Let me make sure I substitute correctly. So, A is 2
===


Now, let's add the real parts and the imaginary parts separately.

Real parts: 2 (from A) + (-4) (from O) + 0 (from P) + 2 (from S) = 0

Imaginary parts: 1 (from A) + 0 (from O) + (-1) (from P) + 4 (from S) = 4

So, putting it all together, the expression simplifies to 0 + 4i, which is just 4i.

Wait, hold on, that doesn't seem right. Let me check my steps again.

Wa

In [71]:
correct_dataset[dataset_pos]

{'problem': 'Suppose that $f$ is a function and $f^{-1}$ is the inverse of $f$.  If $f(1)=2$, $f(2) = 6$, and $f(3)=5$, then what is $f^{-1}(f^{-1}(6))$?',
 'solution': 'Since $f(2) = 6$, we have $f^{-1}(6)=2$. (Note that the hypothesis that $f$ has an inverse implies that there are no other values of $x$ with $f(x) = 6$.)  Similarly, $f(1) =2$ implies $f^{-1}(2)=1$.   So $f^{-1}(f^{-1}(6))=f^{-1}(2)=\\boxed{1}$.',
 'answer': '1',
 'subject': 'Algebra',
 'level': 4,
 'unique_id': 'test/algebra/1199.json',
 'model_answer': "Okay, so I have this problem here: There's a function f, and its inverse function is f inverse. I know that f(1) = 2, f(2) = 6, and f(3) = 5. I need to find f inverse of f inverse of 6, which is written as f^{-1}(f^{-1}(6)). Hmm, okay. Let me think about how to approach this.\n\nFirst, let me recall what an inverse function does. If f is a function that maps an input to an output, then the inverse function f^{-1} maps the output back to the input. So, basically, if f

In [73]:
next_true_tokens.shape[1]

200

In [7]:
generated_ids_compressed.shape[1], next_true_tokens.shape[
    1
], generated_ids_compressed.shape[1] + compressed_part.shape[1], torch.tensor(
    generated_ids
).shape

(1008, 200, 1108, torch.Size([1, 2356]))

In [10]:
correct_dataset[0]['model_answer']

"Okay, so I have this problem here where I need to find the value of A minus O plus P plus S. The given values are A = 2 + i, O = -4, P = -i, and S = 2 + 4i. Hmm, let me write that down to make it clearer.\n\nFirst, let me write down each term:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nAnd the expression I need to compute is A - O + P + S. Hmm, okay. So, I think I need to substitute these values into the expression and then simplify it step by step.\n\nLet me start by substituting the values:\n\nA - O + P + S = (2 + i) - (-4) + (-i) + (2 + 4i)\n\nWait, hold on. Let me make sure I substitute correctly. So, A is 2 + i, O is -4, P is -i, and S is 2 + 4i. So, replacing each term:\n\n= (2 + i) - (-4) + (-i) + (2 + 4i)\n\nOkay, that looks right. Now, let me simplify this step by step. I think I should handle the operations one at a time.\n\nFirst, let's handle the subtraction of O, which is -(-4). Subtracting a negative is the same as adding a positive, so:\n\n(2 + i) - (-4) = (2 + i

In [11]:
tokenizer.decode(next_true_tokens[-1])

'Okay, so I have this problem here where I need to find the value of A minus O plus P plus S. The given values are A = 2 + i, O = -4, P = -i, and S = 2 + 4i. Hmm, let me write that down to make it clearer.\n\nFirst, let me write down each term:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nAnd the expression I need to compute is A - O + P + S. Hmm, okay. So, I think I need to substitute these values into the expression and then simplify it step by step.\n\nLet me start by substituting the values:\n\nA - O + P + S = (2 + i) - (-4) + (-i) + (2 + 4i)\n\nWait, hold on. Let me make sure I substitute correctly. So, A is 2'

In [ ]:
# original
# "Okay, so I have this problem here where I need to find the value of A minus O plus P plus S. The given values are A = 2 + i, O = -4, P = -i, and S = 2 + 4i. Hmm, let me write that down to make it clearer.\n\nFirst, let me write down each term:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nAnd the expression I need to compute is A - O + P + S. Hmm, okay. So, I think I need to substitute these values into the expression and then simplify it step by step.\n\nLet me start by substituting the values:\n\nA - O + P + S = (2 + i) - (-4) + (-i) + (2 + 4i)\n\nWait, hold on. Let me make sure I substitute correctly. So, A is 2 + i, O is -4, P is -i, and S is 2 + 4i. So, replacing each term:\n\n= (2 + i) - (-4) + (-i) + (2 + 4i)\n\nOkay, that looks right. Now, let me simplify this step by step. I think I should handle the operations one at a time.\n\nFirst, let's handle the subtraction of O, which is -(-4). Subtracting a negative is the same as adding a positive, so:\n\n(2 + i) - (-4) = (2 + i) + 4\n\nSo, that simplifies to 2 + 4 + i, which is 6 + i.\n\nAlright, so now the expression becomes:\n\n6 + i + (-i) + (2 + 4i)\n\nWait, let me write that again:\n\n= (6 + i) + (-i) + (2 + 4i)\n\nNow, let's combine the real parts and the imaginary parts separately.\n\nFirst, the real parts: 6, 0 (since there's no real part in -i), and 2.\n\nSo, 6 + 0 + 2 = 8.\n\nNow, the imaginary parts: i, -i, and 4i.\n\nSo, let's add those up:\n\ni - i + 4i\n\nHmm, i minus i is 0, so that leaves us with 4i.\n\nSo, putting it all together, the real part is 8 and the imaginary part is 4i, so the result is 8 + 4i.\n\nWait, let me double-check that. Maybe I made a mistake in the signs somewhere.\n\nStarting over:\n\nA - O + P + S\n\n= (2 + i) - (-4) + (-i) + (2 + 4i)\n\nFirst, handle the subtraction:\n\n(2 + i) - (-4) = 2 + i + 4 = 6 + i\n\nThen, add P: 6 + i + (-i) = 6 + i - i = 6\n\nThen, add S: 6 + (2 + 4i) = 8 + 4i\n\nYes, that seems consistent. So, the final result is 8 + 4i.\n\nWait, hold on, let me make sure I didn't skip any steps or make a mistake in the signs.\n\nSo, starting over:\n\nA - O + P + S\n\n= (2 + i) - (-4) + (-i) + (2 + 4i)\n\nFirst, compute (2 + i) - (-4):\n\nSubtracting a negative is adding, so 2 + i + 4 = 6 + i.\n\nThen, add P, which is -i:\n\n6 + i + (-i) = 6 + (i - i) = 6 + 0 = 6.\n\nThen, add S, which is 2 + 4i:\n\n6 + (2 + 4i) = (6 + 2) + 4i = 8 + 4i.\n\nYes, that seems correct. So, the final answer is 8 + 4i.\n\nAlternatively, maybe I can approach this problem by grouping the real and imaginary parts differently.\n\nLet me try that.\n\nGiven:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nSo, A - O + P + S can be rewritten as:\n\n(2 + i) - (-4) + (-i) + (2 + 4i)\n\nLet me group the real numbers and the imaginary numbers separately.\n\nReal parts: 2, 0 (since O is -4, which is real), -0 (since P is -i, which is imaginary), and 2.\n\nImaginary parts: 1 (from A), 0 (from O), -1 (from P), and 4 (from S).\n\nSo, real parts: 2 + 0 + 0 + 2 = 4\n\nImaginary parts: 1 + 0 - 1 + 4 = 4\n\nWait, hold on, that doesn't seem right. Because when I grouped them earlier, I had 8 + 4i, but now grouping separately, I get 4 + 4i. That's a discrepancy. Hmm, so I must have made a mistake in my grouping.\n\nWait, let me check again.\n\nWait, no, actually, in the expression A - O + P + S, O is a real number, so it's just -(-4) which is +4. So, in the real parts, we have 2 (from A) + 4 (from -O) + 0 (from P) + 2 (from S). So, 2 + 4 + 2 = 8.\n\nImaginary parts: 1 (from A) + 0 (from -O) -1 (from P) + 4 (from S). So, 1 -1 + 4 = 4.\n\nSo, that gives 8 + 4i, which matches my previous result.\n\nWait, so why did I get 4 + 4i when I grouped them? Because I think I miscounted the real parts.\n\nWait, let me write it step by step.\n\nReal parts:\n\nFrom A: 2\n\nFrom -O: -(-4) = +4\n\nFrom P: 0 (since P is -i, which is purely imaginary)\n\nFrom S: 2\n\nSo, total real parts: 2 + 4 + 0 + 2 = 8\n\nImaginary parts:\n\nFrom A: 1\n\nFrom -O: 0 (since O is real)\n\nFrom P: -1\n\nFrom S: 4\n\nTotal imaginary parts: 1 + 0 -1 + 4 = 4\n\nSo, 8 + 4i. That's correct.\n\nSo, my initial grouping was wrong because I didn't account for all the real parts correctly. So, the correct answer is 8 + 4i.\n\nAlternatively, maybe I can think of it as:\n\nA - O + P + S = A + (-O) + P + S\n\nSo, substituting:\n\n= (2 + i) + (4) + (-i) + (2 + 4i)\n\nThen, combine like terms:\n\nReal parts: 2 + 4 + 2 = 8\n\nImaginary parts: i - i + 4i = 4i\n\nSo, 8 + 4i.\n\nYes, that's consistent.\n\nAlternatively, maybe I can think of it as:\n\nA - O + P + S = (A + P) + (-O + S)\n\nCompute A + P first:\n\nA = 2 + i\n\nP = -i\n\nSo, A + P = 2 + i + (-i) = 2 + 0i = 2\n\nThen, compute -O + S:\n\nO = -4, so -O = 4\n\nS = 2 + 4i\n\nSo, -O + S = 4 + (2 + 4i) = 6 + 4i\n\nNow, add A + P and -O + S:\n\n2 + (6 + 4i) = 8 + 4i\n\nSame result.\n\nSo, regardless of the method, I get 8 + 4i.\n\nTherefore, I think that's the correct answer.\n\nWait, just to make sure, let me compute each step numerically.\n\nCompute A - O:\n\nA = 2 + i\n\nO = -4\n\nSo, A - O = (2 + i) - (-4) = 2 + i + 4 = 6 + i\n\nThen, add P:\n\n6 + i + (-i) = 6 + 0i = 6\n\nThen, add S:\n\n6 + (2 + 4i) = 8 + 4i\n\nYep, same result.\n\nAlternatively, if I compute A - O + P + S step by step:\n\nStart with A: 2 + i\n\nSubtract O: 2 + i - (-4) = 2 + i + 4 = 6 + i\n\nAdd P: 6 + i + (-i) = 6 + 0i = 6\n\nAdd S: 6 + (2 + 4i) = 8 + 4i\n\nSame result.\n\nSo, I think I can confidently say that the answer is 8 + 4i.\n\nJust to recap, the steps are:\n\n1. Substitute each term into the expression.\n\n2. Simplify the expression by handling the operations step by step.\n\n3. Combine like terms (real and imaginary parts separately).\n\n4. Verify the result using different methods to ensure accuracy.\n\nSo, yeah, 8 + 4i is the correct answer.\n\n**Final Answer**\nThe value of \\( A - O + P + S \\) is \\boxed{8 + 4i}.\n</think>\n\nGiven the values \\( A = 2 + i \\), \\( O = -4 \\), \\( P = -i \\), and \\( S = 2 + 4i \\), we need to find the value of \\( A - O + P + S \\).\n\nFirst, substitute the given values into the expression:\n\n\\[\nA - O + P + S = (2 + i) - (-4) + (-i) + (2 + 4i)\n\\]\n\nSimplify each term step by step:\n\n1. Subtract \\( O \\) from \\( A \\):\n   \\[\n   (2 + i) - (-4) = 2 + i + 4 = 6 + i\n   \\]\n\n2. Add \\( P \\) to the result:\n   \\[\n   6 + i + (-i) = 6 + 0i = 6\n   \\]\n\n3. Add \\( S \\) to the result:\n   \\[\n   6 + (2 + 4i) = 8 + 4i\n   \\]\n\nThus, the value of \\( A - O + P + S \\) is \\(\\boxed{8 + 4i}\\)."

# compressed
# 'Okay, so I have this problem here where I need to find the value of A minus O plus P plus S. The given values are A = 2 + i, O = -4, P = -i, and S = 2 + 4i. Hmm, let me write that down to make it clearer.\n\nFirst, let me write down each term:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nAnd the expression I need to compute is A - O + P + S. Hmm, okay. So, I think I need to substitute these values into the expression and then simplify it step by step.\n\nLet me start by substituting the values:\n\nA - O + P + S = (2 + i) - (-4) + (-i) + (2 + 4i)\n\nWait, hold on. Let me make sure I substitute correctly. So, A is 2' "
# generated
# \n\nNow, let's add the real parts and the imaginary parts separately.\n\nReal parts: 2 (from A) + (-4) (from O) + 0 (from P) + 2 (from S) = 0\n\nImaginary parts: 1 (from A) + 0 (from O) + (-1) (from P) + 4 (from S) = 4\n\nSo, putting it all together, the expression simplifies to 0 + 4i, which is just 4i.\n\nWait, hold on, that doesn't seem right. Let me check my steps again.\n\nWait, when I separated the real and imaginary parts, I think I might have made a mistake in the signs.\n\nLet me try that again.\n\nStarting over:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nExpression: A - O + P + S\n\nSubstitute the values:\n\n(2 + i) - (-4) + (-i) + (2 + 4i)\n\nNow, distribute the negative sign to O:\n\n2 + i + 4 - i + (-i) + 2 + 4i\n\nWait, hold on, that doesn't seem right. Let me do it step by step.\n\nFirst, A - O: (2 + i) - (-4) = 2 + i + 4 = 6 + i\n\nThen, add P: 6 + i + (-i) = 6 + 0i = 6\n\nThen, add S: 6 + (2 + 4i) = 8 + 4i\n\nWait, that makes more sense. So, the expression simplifies to 8 + 4i.\n\nWait, so earlier I messed up the distribution. Let me correct that.\n\nSo, starting over:\n\nExpression: A - O + P + S\n\nSubstitute:\n\n(2 + i) - (-4) + (-i) + (2 + 4i)\n\nFirst, handle A - O:\n\n(2 + i) - (-4) = 2 + i + 4 = 6 + i\n\nThen, add P:\n\n6 + i + (-i) = 6 + 0i = 6\n\nThen, add S:\n\n6 + (2 + 4i) = 8 + 4i\n\nSo, the final result is 8 + 4i.\n\nWait, so earlier I thought it was 4i, but that was incorrect. I must have made a mistake in distributing the negative sign or something.\n\nSo, the correct answer is 8 + 4i.\n\nBut let me verify once more.\n\nCompute A - O + P + S:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nSo, A - O + P + S = (2 + i) - (-4) + (-i) + (2 + 4i)\n\nCompute each operation step by step:\n\nFirst, (2 + i) - (-4) = 2 + i + 4 = 6 + i\n\nThen, add P: 6 + i + (-i) = 6 + 0i = 6\n\nThen, add S: 6 + (2 + 4i) = 8 + 4i\n\nYes, that seems correct. So, the final result is 8 + 4i.\n\nI think I initially made a mistake in the distribution step, but upon redoing it, I arrived at the correct answer.\n\n**Final Answer**\nThe value of \\( A - O + P + S \\) is \\boxed{8 + 4i}.\n</think>\n\nGiven the values \\( A = 2 + i \\), \\( O = -4 \\), \\( P = -i \\), and \\( S = 2 + 4i \\), we need to find the value of \\( A - O + P + S \\).\n\nStarting with the expression:\n\\[ A - O + P + S \\]\n\nSubstitute the given values:\n\\[ (2 + i) - (-4) + (-i) + (2 + 4i) \\]\n\nSimplify each operation step by step:\n1. Compute \\( (2 + i) - (-4) \\):\n   \\[ 2 + i + 4 = 6 + i \\]\n2. Add \\( P \\):\n   \\[ 6 + i + (-i) = 6 + 0i = 6 \\]\n3. Add \\( S \\):\n   \\[ 6 + (2 + 4i) = 8 + 4i \\]\n\nThus, the value of \\( A - O + P + S \\) is \\(\\boxed{8 + 4i}\\).<｜end▁of▁sentence｜>"

In [12]:
tokenizer.decode(generated_ids[-1])

"Okay, so I have this problem here where I need to find the value of A minus O plus P plus S. The given values are A = 2 + i, O = -4, P = -i, and S = 2 + 4i. Hmm, let me write that down to make it clearer.\n\nFirst, let me write down each term:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nAnd the expression I need to compute is A - O + P + S. Hmm, okay. So, I think I need to substitute these values into the expression and then simplify it step by step.\n\nLet me start by substituting the values:\n\nA - O + P + S = (2 + i) - (-4) + (-i) + (2 + 4i)\n\nWait, hold on. Let me make sure I substitute correctly. So, A is 2 + i, O is -4, P is -i, and S is 2 + 4i. So, replacing each term:\n\n= (2 + i) - (-4) + (-i) + (2 + 4i)\n\nOkay, that looks right. Now, let me simplify this step by step. I think I should handle the operations one at a time.\n\nFirst, let's handle the subtraction of O, which is -(-4). Subtracting a negative is the same as adding a positive, so:\n\n(2 + i) - (-4) = (2 + i

In [13]:
tokenizer.decode(generated_ids_compressed[-1])

"\n\nNow, let's add the real parts and the imaginary parts separately.\n\nReal parts: 2 (from A) + (-4) (from O) + 0 (from P) + 2 (from S) = 0\n\nImaginary parts: 1 (from A) + 0 (from O) + (-1) (from P) + 4 (from S) = 4\n\nSo, putting it all together, the expression simplifies to 0 + 4i, which is just 4i.\n\nWait, hold on, that doesn't seem right. Let me check my steps again.\n\nWait, when I separated the real and imaginary parts, I think I might have made a mistake in the signs.\n\nLet me try that again.\n\nStarting over:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nExpression: A - O + P + S\n\nSubstitute the values:\n\n(2 + i) - (-4) + (-i) + (2 + 4i)\n\nNow, distribute the negative sign to O:\n\n2 + i + 4 - i + (-i) + 2 + 4i\n\nWait, hold on, that doesn't seem right. Let me do it step by step.\n\nFirst, A - O: (2 + i) - (-4) = 2 + i + 4 = 6 + i\n\nThen, add P: 6 + i + (-i) = 6 + 0i = 6\n\nThen, add S: 6 + (2 + 4i) = 8 + 4i\n\nWait, that makes more sense. So, the expression simp

In [19]:
end_think = tokenizer.encode("</think>", add_special_tokens=False)[0]
generated_ids[-1].index(end_think)

1512

In [ ]:
print(tokenizer.decode(generated_ids_compressed[-1]))

In [6]:
tokenizer.decode(next_true_tokens[-1])

"Okay, so I have this problem here where an elephant and a lion are one mile apart. The elephant is running directly away from the lion at 19 miles per hour, and the lion is running towards the elephant at 24 miles per hour. I need to figure out how many minutes it will take for the lion to catch the elephant. Hmm, sounds like a relative speed problem. Let me think about how to approach this.\n\nFirst, let me visualize the situation. There's a lion and an elephant, one mile apart. The lion is running towards the elephant, and the elephant is running away from the lion. Since both are moving towards or away from each other, their speeds will affect the time it takes for the lion to catch up.\n\nI remember that when two objects are moving towards each other, their relative speed is the sum of their individual speeds. But in this case, the lion is moving towards the elephant, and the elephant is moving away from the lion, so"

In [8]:
tokenizer.decode(generated_ids_compressed[-1])

"\n\nNow, let's add the real parts and the imaginary parts separately.\n\nReal parts: 2 (from A) + (-4) (from O) + 0 (from P) + 2 (from S) = 0\n\nImaginary parts: 1 (from A) + 0 (from O) + (-1) (from P) + 4 (from S) = 4\n\nSo, putting it all together, the expression simplifies to 0 + 4i, which is just 4i.\n\nWait, hold on, that doesn't seem right. Let me check my steps again.\n\nWait, when I separated the real and imaginary parts, I think I might have made a mistake in the signs.\n\nLet me try that again.\n\nStarting over:\n\nA = 2 + i\n\nO = -4\n\nP = -i\n\nS = 2 + 4i\n\nExpression: A - O + P + S\n\nSubstitute the values:\n\n(2 + i) - (-4) + (-i) + (2 + 4i)\n\nNow, distribute the negative sign to O:\n\n2 + i + 4 - i + (-i) + 2 + 4i\n\nWait, hold on, that doesn't seem right. Let me do it step by step.\n\nFirst, A - O: (2 + i) - (-4) = 2 + i + 4 = 6 + i\n\nThen, add P: 6 + i + (-i) = 6 + 0i = 6\n\nThen, add S: 6 + (2 + 4i) = 8 + 4i\n\nWait, that makes more sense. So, the expression simp

In [50]:
compressed_part.shape

torch.Size([1, 200, 1536])

In [55]:
print(tokenizer.decode(torch.tensor(generated_ids[-1])))

Okay, so I have this problem where an elephant and a lion are 1 mile apart. The elephant is running directly away from the lion at 19 miles per hour, and the lion is running towards the elephant at 24 miles per hour. I need to figure out how many minutes it will take for the lion to catch the elephant. Hmm, let me think about how to approach this.

First, I know that both the elephant and the lion are moving towards or away from each other. The elephant is moving away at 19 mph, and the lion is moving towards it at 24 mph. So, their relative speed is the difference between their speeds because they're moving towards each other. Wait, no, actually, since the elephant is moving away, the lion has to cover the distance that the elephant is moving away plus the distance the elephant covers while the lion is moving towards it.

Let me clarify. The lion is moving towards the elephant at 24 mph, and the elephant is moving away at 19 mph. So, the lion is closing the gap at a rate of 24 mph min

In [ ]:
tokenizer.decode(torch.tensor(generated_ids[-1]))

"Okay, so I have this problem here where an elephant and a lion are one mile apart. The elephant is running directly away from the lion at 19 miles per hour, and the lion is running towards the elephant at 24 miles per hour. I need to figure out how many minutes it will take for the lion to catch the elephant. Hmm, sounds like a relative speed problem. Let me think about how to approach this.\n\nFirst, let me visualize the situation. There's a lion and an elephant, one mile apart. The lion is running towards the elephant, and the elephant is running away from the lion. Since both are moving towards or away from each other, their speeds will affect the time it takes for the lion to catch up.\n\nI remember that when two objects are moving towards each other, their relative speed is the sum of their individual speeds. But in this case, the lion is moving towards the elephant, and the elephant is moving away from the lion, so their speeds are in opposite directions. Wait, no, actually, the

: 

In [9]:
next_true_tokens.shape

torch.Size([1, 400])

In [39]:
start_embed.shape

torch.Size([1, 1, 1536])

In [38]:
new_embeds_for_compression.shape

torch.Size([4, 2, 224256])

In [ ]:
from hidden_capacity_reasoning.utils import WINDOW_SIZE, VISION_START, VISION_END
from transformers.cache_utils import DynamicCache


def _crop_past_key_values(model, past_key_values, max_length):
    """Crops the past key values up to a certain maximum length."""
    new_past = []
    if model.config.is_encoder_decoder:
        for idx in range(len(past_key_values)):
            new_past.append(
                (
                    past_key_values[idx][0][:, :, :max_length, :],
                    past_key_values[idx][1][:, :, :max_length, :],
                    past_key_values[idx][2],
                    past_key_values[idx][3],
                )
            )
        past_key_values = tuple(new_past)
    # gptbigcode is special and stores kv in shape (batch_size, seq_len, dim), if it's a multi_query model
    elif "gptbigcode" in model.__class__.__name__.lower() or (
        model.config.architectures is not None
        and "gptbigcode" in model.config.architectures[0].lower()
    ):
        if model.config.multi_query:
            for idx in range(len(past_key_values)):
                past_key_values[idx] = past_key_values[idx][:, :max_length, :]
        else:
            for idx in range(len(past_key_values)):
                past_key_values[idx] = past_key_values[idx][:, :, :max_length, :]
    elif isinstance(past_key_values, DynamicCache):
        past_key_values.crop(max_length)
    elif past_key_values is not None:
        for idx in range(len(past_key_values)):
            if past_key_values[idx] != ([], []):
                new_past.append(
                    (
                        past_key_values[idx][0][:, :, :max_length, :],
                        past_key_values[idx][1][:, :, :max_length, :],
                    )
                )
            else:
                new_past.append((past_key_values[idx][0], past_key_values[idx][1]))
        past_key_values = tuple(new_past)
    return past_key_values


# model = trainer.model
generated_tokens = tokenizer.apply_chat_template(
    [
        # {"role": "user", "content": "how many wings has a bird?"},
        {"role": "user", "content": dataset["test"].to_list()[:5][0]["question"]},
    ],
    tokenize=True,
    add_generation_prompt=True,
)

with torch.no_grad(), torch.autocast(device_type="cuda"):
    start_embed = model.base_model.embed_pooler.model.get_input_embeddings()(
        torch.tensor([[VISION_START]], device="cuda")
    )
    end_embed = model.base_model.embed_pooler.model.get_input_embeddings()(
        torch.tensor([[VISION_END]], device="cuda")
    )
    generated_tokens = torch.tensor(generated_tokens).unsqueeze(0).cuda()
    generated_embeds = model.get_input_embeddings()(generated_tokens)
    temp_gen_size = 0
    window_size = WINDOW_SIZE  # + 1
    # new_tokens = 4
    new_tokens = 1
    generation_started = False
    max_steps = (new_tokens + window_size) * 15
    past_key_values_big = None
    print("generated_embeds", generated_embeds.shape)
    for step in range(max_steps):
        if temp_gen_size == window_size + new_tokens:
            # print(
            #     "TOKENS FOR EMDED",
            #     tokenizer.decode(
            #         generated_tokens[:, -(window_size + new_tokens) :][:, :WINDOW_SIZE]
            #         .cpu()
            #         .tolist()[0]
            #     ),
            # )
            # tokenizer.decode(generated_tokens[:, : -window_size ].cpu().tolist()[0])
            if hasattr(model.base_model, "embed_pooler"):
                new_embeds_for_compression = (
                    model.base_model.embed_pooler.model.get_input_embeddings()(
                        generated_tokens[:, -(window_size + new_tokens) :][
                            :, :WINDOW_SIZE
                        ]
                    )
                ).to(torch.bfloat16)
                compressed_part = model.base_model.embed_pooler(
                    new_embeds_for_compression
                )
            else:
                compressed_part = model.embed_pooler(new_embeds_for_compression)
            # gen_embeds_prev = generated_tokens.shape[1]
            if generation_started:
                # past_key_values_big = _crop_past_key_values(
                #     model=model,
                #     past_key_values=past_key_values_big,
                #     max_length=generated_embeds.shape[1] - new_tokens - 2,
                # )
                generated_embeds = torch.cat(
                    [
                        generated_embeds[:, : -(window_size + new_tokens + 1)],
                        # generated_embeds[:, : -(window_size + new_tokens)],
                        compressed_part,
                        # torch.randn(1, 1, 1536, device="cuda"),
                        end_embed,
                        generated_embeds[:, -new_tokens:],
                    ],
                    dim=1,
                )
            else:
                # past_key_values_big = _crop_past_key_values(
                #     model=model,
                #     past_key_values=past_key_values_big,
                #     max_length=generated_embeds.shape[1] - new_tokens - 3,
                # )
                generated_embeds = torch.cat(
                    [
                        generated_embeds[:, : -(window_size + new_tokens)],
                        start_embed,
                        # torch.randn(1, 1, 1536, device="cuda"),
                        compressed_part,
                        end_embed,
                        generated_embeds[:, -new_tokens:],
                    ],
                    dim=1,
                )
                generation_started = True
            past_key_values_big = _crop_past_key_values(
                model=model,
                past_key_values=past_key_values_big,
                max_length=generated_embeds.shape[1] - new_tokens - 2,
            )
            temp_gen_size = 1

        outputs = model(
            inputs_embeds=generated_embeds,
            past_key_values=past_key_values_big,
            # use_cache=False,
        )
        logits = outputs.logits
        past_key_values_big = outputs.past_key_values
        top_token = logits.argmax(-1)[-1][-1]
        top_token_embed = model.get_input_embeddings()(top_token)
        # print(top)
        generated_tokens = torch.cat([generated_tokens, top_token.reshape(1, 1)], dim=1)

        generated_embeds = torch.cat(
            [generated_embeds, top_token_embed.reshape(1, 1, -1)], dim=1
        )
        # print(temp_gen_size, tokenizer.decode(generated_tokens[-1]))

        temp_gen_size += 1

print(tokenizer.decode(generated_tokens[-1]))

# break

generated_embeds torch.Size([1, 62, 1536])
<｜begin▁of▁sentence｜><｜User｜>Here's a question: What do many people believe happens after you die?  Here are possible answers to this question: - stop moving - nothing - go to heaven - stop living - stop breathing  I believe the correct choice is "go to heaven", here's why:
Answer:<｜Assistant｜><think>
Okay, so I'm trying to figure out the answer is correct. Let me think about it again. I think the user is trying to figure out the correct answer to the question they're asking. They provided a list of answers, and I need to heaven, but I'm not sure if I'm sure if I'm on the right track. Let me break it down step by step. The question is about what people believe happens after you die. I know that when someone dies, they believe that people often believe in something called the afterlife, which is the belief that after you die, you don't need to move or anything else. So, the correct answer is "go to heaven", which is the correct answer. The othe

# я не знаю, может быть я неправильно управляюсь с KV-cache

In [18]:
generated_tokens.shape

torch.Size([1, 227])

In [35]:
new_embeds_for_compression.shape

torch.Size([1, 10, 1536])

In [34]:
[1, 2, 3, 4, 5, 6, 7, 8][:-2]

[1, 2, 3, 4, 5, 6]

In [ ]:
print(dataset["test"][0]["answer"])

In [29]:
generated_embeds.shape

torch.Size([1, 100, 1536])

In [28]:
generated_tokens.shape

torch.Size([1, 227])

### Тестируем на MATH-500

In [82]:
len(correct_dataset)

155

In [ ]:
from transformers import AutoTokenizer
from hidden_capacity_reasoning.utils import (
    WINDOW_SIZE,
    VISION_START,
    VISION_END,
    EOS_TOKEN_ID,
)
import torch
import json
from lm_eval.tasks.hendrycks_math.utils import strip_string, remove_boxed, is_equiv
from hidden_capacity_reasoning.evaluation.math_500.utils import (
    dataset_answer_filter,
    model_answer_filter,
)
from tqdm.notebook import tqdm

correct_items = 0
torch.manual_seed(0)
base_prompt = open(
    "hidden_capacity_reasoning/evaluation/math_500/math_500_prompt"
).read()
tokenizer = AutoTokenizer.from_pretrained(model_name)

evaluation_dataset = []

for dataset_pos in tqdm(range(len(correct_dataset))):
    # dataset_pos = 11
    input_ids = correct_dataset[dataset_pos]["problem"]
    # print(input_ids)
    # print("===")
    # print("===")

    input_ids = [
        tokenizer.apply_chat_template(
            [
                {
                    "role": "user",
                    "content": base_prompt.format(question=input_ids),
                },
            ],
            tokenize=True,
            add_generation_prompt=True,
        )
    ]

    # input_ids = dataset_item["input_ids"]
    # generated_ids = dataset_item["generated_ids"]
    generated_ids = [
        tokenizer.encode(
            correct_dataset[dataset_pos]["model_answer"],
            add_special_tokens=False,
        )
    ]

    device = "cuda"

    with torch.no_grad():
        start_embed = model.base_model.embed_pooler.model.get_input_embeddings()(
            torch.tensor([[VISION_START]], device="cuda")
        )
        end_embed = model.base_model.embed_pooler.model.get_input_embeddings()(
            torch.tensor([[VISION_END]], device="cuda")
        )
        input_ids = torch.tensor(input_ids).cuda()
        input_ids_embeds = model.get_input_embeddings()(input_ids)
        # windows_amount = 100
        # windows_amount = 200
        # windows_amount = 300
        windows_amount = 400
        # windows_amount = 500
        # windows_amount = 2
        generated_tokens_amount = WINDOW_SIZE * windows_amount
        original_total_len = torch.tensor(generated_ids).shape[1]
        if generated_tokens_amount > original_total_len:
            windows_amount = original_total_len // WINDOW_SIZE
            generated_tokens_amount = WINDOW_SIZE * windows_amount

        next_true_tokens = torch.tensor(generated_ids, device="cuda")[
            :, :generated_tokens_amount
        ]

        # next_true_tokens = torch.tensor(next_true_tokens, device="cuda")

        original_embeds = (model.get_input_embeddings()(next_true_tokens)).to(
            torch.bfloat16
        )

        # compressed_part = model.base_model.embed_pooler(new_embeds_for_compression)
        new_embeds_for_compression = original_embeds.reshape(
            windows_amount, WINDOW_SIZE, -1
        )
        compressed_part = model.base_model.embed_pooler(new_embeds_for_compression)
        compressed_part = compressed_part.reshape(1, windows_amount, -1)
        # start_embed = torch.rand_like(start_embed)
        # compressed_part = torch.rand_like(compressed_part)
        # end_embed = torch.rand_like(end_embed)
        generated_embeds = torch.cat(
            [
                input_ids_embeds,
                start_embed,
                compressed_part,
                end_embed,
            ],
            dim=1,
        )
        # print("COMPRESSED PART", tokenizer.decode(next_true_tokens[-1]))
        # print("===")
        generated_ids_compressed = model.generate(
            inputs_embeds=generated_embeds,
            # attention_mask=torch.ones_like(generated_embeds),
            attention_mask=torch.ones(
                generated_embeds.shape[:2],
                device="cuda",
            ).long(),
            max_new_tokens=4096,
            # max_new_tokens=5,
            do_sample=False,
            # do_sample=True,
            # temperature=0.6,
            # top_p=0.95,
        )
        # break
    generated_result = tokenizer.decode(generated_ids_compressed[-1])
    # print()
    gold_answer = correct_dataset[dataset_pos]["answer"]
    answer = dataset_answer_filter(gold_answer)
    model_answer = model_answer_filter(generated_result)
    if is_equiv(answer, model_answer):
        correct_items += 1
        print("CORRECT")
    else:
        print("WRONG", gold_answer)
        print(generated_result)
    compressed_total_len = generated_ids_compressed.shape[1] + compressed_part.shape[1]
    print(
        generated_ids_compressed.shape[1],
        next_true_tokens.shape[1],
        compressed_total_len,
        torch.tensor(generated_ids).shape[1],
    )
    evaluation_dataset.append(
        {
            **correct_dataset[dataset_pos],
            "compressed_input_part": tokenizer.decode(next_true_tokens[-1]),
            "compressed_output_generation": generated_result,
            "compressed_compression_size": generated_tokens_amount,
            "original_total_len": torch.tensor(generated_ids).shape[1],
            "compressed_total_len": compressed_total_len,
        }
    )
    print("===")
    print("===")
    print("===")
    # break
    # embeds_generation_tokens = generated_tokens[-1]

  0%|          | 0/136 [00:00<?, ?it/s]

WRONG 12
 1 mile.

Let me define the position of the lion as a function of time: \( P_{lion}(t) = 0 + 24t \).

The position of the elephant, moving away from the lion, is \( P_{elephant}(t) = 1 - 19t \).

They will meet when \( P_{lion}(t) = P_{elephant}(t) \).

So, setting them equal:

\( 24t = 1 - 19t \)

Combine like terms:

\( 24t + 19t = 1 \)

\( 43t = 1 \)

\( t = 1 / 43 \) hours.

Wait, hold on, this is different from before. Earlier, I thought it was 1/5 hours, which is 12 minutes, but now I'm getting 1/43 hours. Which one is correct?

Wait, let me check my equations.

Position of lion: starting at 0, moving towards positive direction at 24 mph. So, \( P_{lion}(t) = 24t \).

Position of elephant: starting at 1, moving away from lion towards negative direction at 19 mph. So, \( P_{elephant}(t) = 1 - 19t \).

Setting them equal: \( 24t = 1 - 19t \).

So, \( 24t + 19t = 1 \)

\( 43t = 1 \)

\( t = 1 / 43 \) hours.

Wait, so this is 1 divided by 43 hours. Let me convert that to min

In [8]:
generated_embeds.shape[:2]

torch.Size([1, 656])

In [12]:
torch.ones(
    generated_embeds.shape[:2],
    device="cuda",
).long()

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1

In [9]:
original_embeds.shape

torch.Size([1, 719, 1536])

In [ ]:
print(tokenizer.decode(next_true_tokens[-1]))

In [4]:
len(correct_dataset) / len(dataset), correct_items / len(dataset)

(0.8908045977011494, 0.7816091954022989)

In [17]:
# model.base_model_prefix

'model'

In [18]:
evaluation_dataset[0]

{'problem': 'Below is a magic square, meaning that the sum of the numbers in each row, in each column, and in each of the $2$ main diagonals are equal. What is the value of $n$?\n\n[asy]size(125);\nfor(int i = 0; i<4; ++i)\n{\n\ndraw((0,i)--(3,i),linewidth(1));\n}\n\nfor(int j = 0; j<4; ++j)\n{\n\ndraw((j,0)--(j,3),linewidth(1));\n}\n\nlabel("$n-3$",(.5,.5));\nlabel("3",(.5,1.5));\nlabel("$n+1$",(.5,2.5));\n\nlabel("$n+2$",(1.5,.5));\nlabel("$2n-9$",(1.5,1.5));\nlabel("$1$",(1.5,2.5));\n\nlabel("$2$",(2.5,.5));\nlabel("$n$",(2.5,1.5));\nlabel("$n-1$",(2.5,2.5));\n[/asy]',
 'solution': 'First, we can evaluate the sum across the first row, which gives $(n+1)+1+(n-1)=2n+1$.  Evaluate the sum of the entries across the second row, $3+(2n-9)+n=3n-6$. Now, since we have a magic square, these two sums are equal.  So $2n+1=3n-6$. Isolating $n$, we obtain $n = \\boxed{7}$.\n\nThe square will look like: [asy] size(2cm);\ndraw((0,0)--(3,0)--(3,3)--(0,3)--cycle,linewidth(1));\ndraw((1,0)--(1,3),lin

In [5]:
import json

base_path = "hidden_capacity_reasoning/evaluation/math_500/evals/compression_tests"
save_id = f"test_correct_dataset_{len(correct_dataset)}_compressed_window_amount=400,window=2__2025_04_22_01_43_43_347583__10000"
with open(f"{base_path}/{save_id}.json", "w") as f:
    json.dump(evaluation_dataset, f)

In [27]:
import json

with open(
    f"hidden_capacity_reasoning/evaluation/math_500/evals/compression_tests/correct_dataset_155_compressed_window_amount=400,window=2__2025_04_22_01_43_43_347583__90000.json",
    "r",
) as f:
    evaluation_dataset = json.load(f)

In [6]:
original_total_len = 0
compressed_total_len = 0
for item in evaluation_dataset:
    original_total_len += item["original_total_len"]
    compressed_total_len += item["compressed_total_len"]
original_total_len, compressed_total_len, compressed_total_len / original_total_len

(350015, 338450, 0.9669585589189035)